# Image Classification using MLP


This Code Template is for Image Classificaiton task using Multilayer perceptron (MLP). which is a class of feedforward artificial neural network (ANN)

<img src="https://cdn.blobcity.com/assets/gpu_required.png"  height="25" style="margin-bottom:-15px" />

## Coming Soon